In [8]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [12]:
seeds = pd.read_csv(os.path.join(sys.path[0], '../Data/2020DataFiles/2020DataFiles/2020-Mens-Data/MDataFiles_Stage1/MNCAATourneySeeds.csv'))
results = pd.read_csv(os.path.join(sys.path[0], '../Data/2020DataFiles/2020DataFiles/2020-Mens-Data/MDataFiles_Stage1/MNCAATourneyCompactResults.csv'))
regSeasResults = pd.read_csv(os.path.join(sys.path[0], '../Data/2020DataFiles/2020DataFiles/2020-Mens-Data/MDataFiles_Stage1/MRegularSeasonCompactResults.csv'))


In [3]:
print('')
print(seeds[0:5])
print(results[0:5])


   Season Seed  TeamID
0    1985  W01    1207
1    1985  W02    1210
2    1985  W03    1228
3    1985  W04    1260
4    1985  W05    1374
   Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1985     136     1116      63     1234      54    N      0
1    1985     136     1120      59     1345      58    N      0
2    1985     136     1207      68     1250      43    N      0
3    1985     136     1229      58     1425      55    N      0
4    1985     136     1242      49     1325      38    N      0


In [4]:
seeds['TeamID'] = pd.Categorical(seeds.TeamID)
seeds['section'] = seeds.Seed.str.extract(r'(.)')
seeds['numSeed'] = seeds.Seed.str.extract(r'([0-9][0-9])')
seeds['playInRank'] = seeds.Seed.str.extract(r'[0-9][0-9](.?)')

seeds['playInRank'].replace(to_replace = ['a', 'b', ''], value = [1, 2, 0], inplace = True)
seeds['playInRank'] = seeds.playInRank.astype(float)

seeds['numSeed'] = seeds.numSeed.astype(float)

seeds['numSeed'] = seeds.numSeed + (seeds.playInRank / 10)

seeds.drop(['Seed', 'playInRank'], axis = 1, inplace = True)

In [ ]:
seeds.to_csv('test.csv', index = False)

In [5]:
results['WTeamID'] = pd.Categorical(results.WTeamID)
results['LTeamID'] = pd.Categorical(results.LTeamID)

In [6]:
seeds.rename(columns = {'TeamID': 'WTeamID'}, inplace = True)
seedResults = pd.merge(results, seeds, on = ['WTeamID', 'Season'])
seedResults.rename(columns = {'section': 'WSection', 'numSeed': 'WNumSeed'}, inplace = True)

seeds.rename(columns = {'WTeamID': 'LTeamID'}, inplace = True)
seedResults = pd.merge(seedResults, seeds, on = ['LTeamID', 'Season'])
seedResults.rename(columns = {'section': 'LSection', 'numSeed': 'LNumSeed'}, inplace = True)

seeds.rename(columns = {'LTeamID': 'TeamID'}, inplace = True)

In [ ]:
seedResults.to_csv('test.csv', index = False)

In [7]:
nrow = seedResults.shape[0]
seedResults['WExpected'] = seedResults.WTeamID

start = time.time()

for i in range(nrow):
    if seedResults.loc[i, 'WNumSeed'] >= seedResults.loc[i, 'LNumSeed']:
        seedResults.loc[i, 'WExpected'] = seedResults.loc[i, 'LTeamID']

end = time.time()

print(end - start)

seedResults['correctGuess'] = seedResults.WExpected == seedResults.WTeamID

0.43552112579345703


In [ ]:
accuracy = seedResults.correctGuess.sum() / nrow
print("Picking highest seed strategy accuracy: ", accuracy)

In [22]:
start = time.time()

years = regSeasResults.Season.unique()
masterSeasonTotals = pd.DataFrame(columns = ['Season', 'TeamID', 'W', 'L', 'PF', 'PA'])

for year in years[0:2]:
    print(year)
    actSeason = regSeasResults[regSeasResults.Season == year]
    actSeason.reset_index(inplace = True)
    teams = actSeason.WTeamID.append(actSeason.LTeamID).unique()
    nrow = len(teams)
    actSeasonTotals = pd.DataFrame(data = np.zeros((nrow, 6)), columns = ['Season', 'TeamID', 'W', 'L', 'PF', 'PA'])

    actSeasonTotals['Season'] = pd.Series(data = [year] * nrow)
    actSeasonTotals['TeamID'] = pd.Series(data = teams)

    for game in range(actSeason.shape[0]):
        WInd = actSeasonTotals.index[actSeasonTotals['TeamID'] == actSeason.loc[game, 'WTeamID']].tolist()[0]
        LInd = actSeasonTotals.index[actSeasonTotals['TeamID'] == actSeason.loc[game, 'LTeamID']].tolist()[0]
        actSeasonTotals.loc[WInd, 'W'] += 1
        actSeasonTotals.loc[LInd, 'L'] += 1
        actSeasonTotals.loc[WInd, 'PF'] += actSeason.loc[game, 'WScore']
        actSeasonTotals.loc[WInd, 'PA'] += actSeason.loc[game, 'LScore']
        actSeasonTotals.loc[LInd, 'PF'] += actSeason.loc[game, 'LScore']
        actSeasonTotals.loc[LInd, 'PA'] += actSeason.loc[game, 'WScore']

    masterSeasonTotals = pd.concat([masterSeasonTotals, actSeasonTotals], axis = 0)
        
end = time.time()

print(end - start)

1985
1986
23.480494022369385
